In [1]:
from graph_ds import PyH3Graph

In [2]:
graph = PyH3Graph()
graph.create(osm_path="../resources/copenhagen-with-ways.osm.pbf", gtfs_path="../resources/rejseplanen.zip")

processing osm pbf file: ../resources/copenhagen-with-ways.osm.pbf
converted OSM file into 446019 edges
osm graph created with 364511 nodes in 9 s
getting GTFS feed from ../resources/rejseplanen.zip
routes: 1631
gtfs graph created with 103525 nodes in 36 s
merged graph created with 465889 nodes in 0 s


In [6]:
%%time
dist_result = graph.matrix_bfs(origins=[631049690224658431, 631049690149925887], destinations=[631049690149925887, 631049690224658431])

CPU times: user 550 ms, sys: 0 ns, total: 550 ms
Wall time: 274 ms


In [7]:
dist_result

[[10.896716781999999], [5.8967167819999995]]